In [2]:
# Import necessary packages
import os 
import random 
import string 
import nipype.interfaces.io as nio 
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util 
import matplotlib.pylab as plt 
import nibabel as nb
import numpy as np 

from os.path import abspath 
from nipype import Workflow, Node, Function 
from nipype.interfaces.fsl import ExtractROI, BET, GLM, ErodeImage, ApplyMask, FAST 
from nipype.interfaces.spm import SliceTiming, Realign, Coregister, Normalize, Smooth, ApplyTransform
from nipype.interfaces.spm.utils import Reslice
from nipype.interfaces.afni import Bandpass
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.base import Undefined
from nipype.utils.filemanip import filename_to_list
from nilearn.plotting import plot_anat
from nipype.interfaces import IdentityInterface
from nipype.algorithms.confounds import CompCor, ACompCor, TSNR
from nilearn.signal import clean 
from nipype.interfaces.io import SelectFiles, DataSink
from functions import Design_Matrix, Motion_reg_creator, QC, roi_t_extract, tmp_file_removal, mask_t_extract, detrending

## Initialization

In [3]:
# Initialize paths and parameters here 

# Main directory of the data 
data_dir = "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/Mind_Blanking/data/Neuroimaging/Raw_data"

# Main directory to save results 
out_dir = "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/Mind_Blanking/data/Neuroimaging/Preproc_MNI"

# SPM and FSL main directories
SPM_dir = '/home/sepehr/My_Documents/Softwares/SPM/spm12'
FSL_dir = '/usr/local/fsl'

# fMRI acquisition parameters
Num_of_Slices = 34 
TR = 2.04
TA = TR - TR/float(Num_of_Slices)

# Some Parameters and files
#you should give full permission to the fsl folder (sudo chmod -R a+rwx /path/to/fsl)

num_of_scans_rmv = 201 # Number of initial scans to remove
MNI_template_comp = FSL_dir + '/data/standard/MNI152_T1_2mm_brain.nii.gz'
sig = nb.load(MNI_template_comp)
nb.save(sig, FSL_dir + '/data/standard/MNI152_T1_2mm_brain.nii')
MNI_template = FSL_dir + '/data/standard/MNI152_T1_2mm_brain.nii'
atlas = '/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/GITLAB/mind_blanking/Atlas_files/Schaefer_100par_7net_2mm/atlas.nii'

# List of subjects to apply pipeline on  
subject_list = ['s8211']

## Node Definition 

In [4]:
# Structural Workflow Nodes 

# Brain Extraction 
Bet = Node(BET(), name='BrainExtraction')
Bet.inputs.mask = True 
Bet.inputs.frac = 0.3 
Bet.inputs.output_type = 'NIFTI'

# Segmentation and Bias Correction 
Seg = Node(FAST(), name='Segmentation')
Seg.inputs.number_classes = 3 
Seg.inputs.output_biascorrected = True 
Seg.inputs.img_type = 1 #T1 >> 2 for T2 and 3 for PD
Seg.inputs.segments = True 
Seg.inputs.no_pve = True 
Seg.inputs.probability_maps = False 
Seg.inputs.output_type = 'NIFTI'
pickindex = lambda x, i: x[i]

# Erosion of WM Mask 
erwm = Node(ErodeImage(), name='ErosionWM')
erwm.inputs.kernel_shape = '3D'
erwm.inputs.output_type = 'NIFTI'

# Erosion of CSF Mask 
ercsf = Node(ErodeImage(), name='ErosionCSF')
ercsf.inputs.kernel_shape = '3D'
ercsf.inputs.output_type = 'NIFTI'

# Normalization of GM Mask to MNI Space 
Normgm = Node(Normalize(), name="NormGM")
Normgm.inputs.template = MNI_template
Normgm.inputs.write_bounding_box = [[-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]]
Normgm.inputs.write_interp = 0

# Normalization of WM Mask to MNI Space 
Normwm = Node(Normalize(), name="NormWM")
Normwm.inputs.template = MNI_template
Normwm.inputs.write_bounding_box = [[-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]]
Normwm.inputs.write_interp = 0

# Normalization of CSF Mask to MNI Space 
Normcsf = Node(Normalize(), name="NormCSF")
Normcsf.inputs.template = MNI_template
Normcsf.inputs.write_bounding_box = [[-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]]
Normcsf.inputs.write_interp = 0

# Normalization of Brain Mask to MNI Space 
Normmask = Node(Normalize(), name="NormMask")
Normmask.inputs.template = MNI_template
Normmask.inputs.write_bounding_box = [[-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]]
Normmask.inputs.write_interp = 0

##############################################################################################

# Functional Workflow Nodes 

# Initial Volume Removal 
VolRv = Node(ExtractROI(), name="VolumeRemoval")
VolRv.inputs.t_min = num_of_scans_rmv
VolRv.inputs.t_size = -1
VolRv.inputs.output_type = 'NIFTI'

# Realignment of functional data 
ReAlign = Node(Realign(), name="Realignment")
ReAlign.inputs.quality = 0.9
ReAlign.inputs.fwhm = 5
ReAlign.inputs.separation = 4 
ReAlign.inputs.register_to_mean = True 
ReAlign.inputs.interp = 2
ReAlign.inputs.write_interp = 2
ReAlign.inputs.write_mask = True 

# ART Artifact Detection 
Art = Node(ArtifactDetect(), name="ArtifactDetection")
Art.inputs.parameter_source = 'SPM'
Art.inputs.use_norm = Undefined
Art.inputs.rotation_threshold = 0.05
Art.inputs.translation_threshold = 3 
Art.inputs.zintensity_threshold = 3
Art.inputs.mask_type = 'spm_global'
Art.inputs.save_plot = True 

# Coregistration of functional data to structural data in native space 
CoReg = Node(Coregister(), name='Coregistration')
CoReg.inputs.jobtype = 'estimate'
CoReg.inputs.cost_function = 'nmi'
CoReg.inputs.separation = [4, 2]
CoReg.inputs.tolerance = [0.02, 0.02, 0.02, 
                          0.001, 0.001, 0.001, 
                          0.01, 0.01, 0.01, 
                          0.001, 0.001, 0.001]

# Normalization of Functional Images to MNI Space 
Norm = Node(Normalize(), name="Normalization")
Norm.inputs.template = MNI_template
Norm.inputs.write_bounding_box = [[-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]]

################################################################################################################

# Denoising Workflow Nodes

# Motion and Outlier Regressors Creation 
Mot = Node(Function(input_names=['motion_params', 'outliers', 'N_motion_reg'], 
                   output_names=['output'], function=Motion_reg_creator), name='MotionReg')

# WM and CSF Noise Components Extraction
WMnoise = Node(CompCor(), name='WMReg')
CSFnoise = Node(CompCor(), name='CSFReg')
WMnoise.inputs.repetition_time = TR
CSFnoise.inputs.repetition_time = TR
WMnoise.inputs.num_components = 5
CSFnoise.inputs.num_components = 5
WMnoise.inputs.pre_filter = 'cosine'
CSFnoise.inputs.pre_filter = 'cosine'
WMavg = Node(Function(input_names=['mask', 'img'],
                     output_names=['out_file'], function=mask_t_extract), name='WMavg')
CSFavg = Node(Function(input_names=['mask', 'img'],
                      output_names=['out_file'], function=mask_t_extract), name='CSFavg')

# Create Design Matrix 
DesMat = Node(Function(input_names=['motion_reg', 'wmnoise_reg', 'csfnoise_reg', 'outliers'],
                      output_names=['file_dir'], function=Design_Matrix), name='DesignMat')

# Smooting 
Smth = Node(Smooth(), name='Smoothing')
Smth.inputs.fwhm = [8, 8, 8]
Smth.inputs.implicit_masking = False 

# Detrending 
Det = Node(Function(input_names=['sig', 'mask', 'outliers'], 
                   output_names=['out_addr'], function=detrending), name='Detrend')

# Perform GLM 
glm = Node(GLM(), name='GLM')
glm.inputs.out_res_name = 'glm_res.nii'
glm.inputs.output_type = 'NIFTI'
glm.inputs.demean = False  

# Bandpass Filtering
bpf = Node(Bandpass(), name='BPF')
bpf.inputs.lowpass = 0.09
bpf.inputs.highpass = 0.008
bpf.inputs.tr = TR
bpf.inputs.outputtype = 'NIFTI'

###############################################################################################################

# Input and Output Stream Nodes 

# Reading subjects' structural and functional data (data should be stored properly)

infosource = Node(
    interface=util.IdentityInterface(fields=['subject_id']), name="infosource")
infosource.iterables = ('subject_id', subject_list)

# Input Node 
templates = {
    'anat':data_dir+'/{subject_id}/struct.nii',
    'func':data_dir+'/{subject_id}/func.nii'
}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Output Node
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = out_dir

# ROI Time Extraction 
roiT = Node(Function(input_names=['atlas', 'img', 'sub_id', 'out_dir'], 
                   output_names=['file_dir'], function=roi_t_extract), name='ROI_Time_Extraction')
roiT.inputs.out_dir = out_dir
roiT.inputs.atlas = atlas

# Tmp file removal 
tmpf = Node(Function(input_names=['dsink', 'data_dir', 'subj_id', 'file_dir'], 
                     function=tmp_file_removal), name='Tmp_File_Rmv')
tmpf.inputs.data_dir = data_dir 
tmpf.base_directory = data_dir + '/tmp'

## WorkFlows 

In [5]:
# Structural Workflow 
strct_workflow = Workflow(name='Structural')
strct_workflow.base_dir = data_dir 
strct_workflow.connect(Bet, 'out_file', Seg, 'in_files')
strct_workflow.connect(Seg, 'restored_image', Normwm, 'source')
strct_workflow.connect(Seg, 'restored_image', Normgm, 'source')
strct_workflow.connect(Seg, 'restored_image', Normcsf, 'source')
#strct_workflow.connect(Seg, 'restored_image', Normmask, 'source')
strct_workflow.connect(Seg, ('tissue_class_files', pickindex, 2), Normwm, 'apply_to_files')
strct_workflow.connect(Seg, ('tissue_class_files', pickindex, 1), Normgm, 'apply_to_files')
strct_workflow.connect(Seg, ('tissue_class_files', pickindex, 0), Normcsf, 'apply_to_files')
strct_workflow.connect(Normwm, 'normalized_files', erwm, 'in_file')
strct_workflow.connect(Normcsf, 'normalized_files', ercsf, 'in_file')

#strct_workflow.connect(Bet, 'mask_file', Normmask, 'apply_to_files')

# Functional Workflow 
func_workflow = Workflow(name='Functional')
func_workflow.base_dir = data_dir 
func_workflow.connect(VolRv, 'roi_file', ReAlign, 'in_files')
func_workflow.connect(ReAlign, 'realigned_files', Art, 'realigned_files')
func_workflow.connect(ReAlign, 'realignment_parameters', Art, 'realignment_parameters')
func_workflow.connect(ReAlign, 'mean_image', CoReg, 'source')
func_workflow.connect(ReAlign, 'realigned_files', CoReg, 'apply_to_files')
func_workflow.connect(CoReg, 'coregistered_files', Norm, 'apply_to_files')

#Denoising Workflow 
den_workflow = Workflow(name='Denoising')
den_workflow.base_dir = data_dir 
den_workflow.connect(Mot, 'output', DesMat, 'motion_reg')
den_workflow.connect(WMnoise, 'components_file', DesMat, 'wmnoise_reg')
#den_workflow.connect(WMavg, 'out_file', DesMat, 'wmavg_reg')
den_workflow.connect(CSFnoise, 'components_file', DesMat, 'csfnoise_reg')
#den_workflow.connect(CSFavg, 'out_file', DesMat, 'csfavg_reg')
den_workflow.connect(DesMat, 'file_dir', glm, 'design')
den_workflow.connect(Smth, 'smoothed_files', Det, 'sig')
den_workflow.connect(Det, 'out_addr', glm, 'in_file')
den_workflow.connect(glm, 'out_res', bpf, 'in_file')

# Connect Workflows 
scaffoldflow = Workflow(name='scaffoldflow')
scaffoldflow.base_dir = data_dir 
scaffoldflow.connect([
    (infosource, selectfiles, [('subject_id', 'subject_id')]),
    (selectfiles, strct_workflow, [('anat', 'BrainExtraction.in_file')]),
    (selectfiles, func_workflow, [('func', 'VolumeRemoval.in_file')]),
    (strct_workflow, func_workflow, [('Segmentation.restored_image','Coregistration.target')]),
    (strct_workflow, func_workflow, [('Segmentation.restored_image', 'Normalization.source')]),
    (func_workflow, den_workflow, [('Normalization.normalized_files', 'WMReg.realigned_file')]),
    (func_workflow, den_workflow, [('Normalization.normalized_files', 'CSFReg.realigned_file')]),
    (func_workflow, den_workflow, [('Normalization.normalized_files', 'Smoothing.in_files')]),
    (func_workflow, den_workflow, [('Realignment.realignment_parameters', 'MotionReg.motion_params')]),
    (func_workflow, den_workflow, [('ArtifactDetection.outlier_files', 'MotionReg.outliers')]),
    (func_workflow, den_workflow, [('ArtifactDetection.outlier_files', 'DesignMat.outliers')]),
    (func_workflow, den_workflow, [('ArtifactDetection.outlier_files', 'Detrend.outliers')]),
    (strct_workflow, den_workflow, [('ErosionWM.out_file', 'WMReg.mask_files')]),
    (strct_workflow, den_workflow, [('ErosionCSF.out_file', 'CSFReg.mask_files')]),
    (strct_workflow, den_workflow, [('NormGM.normalized_files', 'Detrend.mask')]),
    (strct_workflow, den_workflow, [('NormGM.normalized_files', 'GLM.mask')]),
    (den_workflow, roiT, [('BPF.out_file', 'img')]),
    (infosource, roiT, [('subject_id', 'sub_id')]),
    #############################################################################
    (infosource, datasink, [('subject_id', 'container')]),
    (func_workflow, datasink, [('VolumeRemoval.roi_file', 'VolumeRemoval.@file')]),
    (func_workflow, datasink, [('Realignment.mean_image', 'Realignment.@mean')]),
    (func_workflow, datasink, [('Realignment.realignment_parameters', 'Realignment.@param')]),
    (func_workflow, datasink, [('Realignment.realigned_files', 'Realignment.@ra_files')]),
    (func_workflow, datasink, [('Coregistration.coregistered_files', 'Coregistration.@coreg')]),
    (func_workflow, datasink, [('Normalization.normalized_source', 'Normalization.@source')]),
    (func_workflow, datasink, [('Normalization.normalization_parameters', 'Normalization.@params')]),
    (func_workflow, datasink, [('Normalization.normalized_files', 'Normalization.@files')]),
    (func_workflow, datasink, [('ArtifactDetection.outlier_files', 'ART.@outliers')]),
    (func_workflow, datasink, [('ArtifactDetection.plot_files', 'ART.@outliers_img')]),
    (func_workflow, datasink, [('ArtifactDetection.displacement_files', 'ART.@displacement')]),
    
    
    (strct_workflow, datasink, [('BrainExtraction.out_file', 'BET.@brain')]),
    (strct_workflow, datasink, [('BrainExtraction.mask_file', 'BET.@mask')]),
    (strct_workflow, datasink, [('Segmentation.tissue_class_files', 'Segmentation.@masks')]),
    (strct_workflow, datasink, [('Segmentation.restored_image', 'Segmentation.@bias_corrected')]),
    (strct_workflow, datasink, [('NormGM.normalized_files', 'Normalization.@gm')]),
    (strct_workflow, datasink, [('ErosionWM.out_file', 'Normalization.@wm')]),
    (strct_workflow, datasink, [('ErosionCSF.out_file', 'Normalization.@csf')]),
    #(strct_workflow, datasink, [('NormMask.normalized_files', 'Normalization.@mask')]),

    (den_workflow, datasink, [('Smoothing.smoothed_files', 'Smoothing.@smoothed')]),
    (den_workflow, datasink, [('DesignMat.file_dir', 'DesignMatrix.@desmat')]),
    (den_workflow, datasink, [('GLM.out_res', 'GLM_Results.@res')]),
    (den_workflow, datasink, [('GLM.out_file', 'GLM_Results.@betas')]),
    (den_workflow, datasink, [('BPF.out_file', 'BPF.@final')]),
    
    (datasink     , tmpf,          [('out_file', 'dsink')]),
    (roiT         , tmpf,          [('file_dir', 'file_dir')]),
    (infosource   , tmpf,          [('subject_id', 'subj_id')])
    
])
scaffoldflow.write_graph(graph2use='orig', format='png', simple_form=True, 
                    dotfilename='../../Results/preproc_pipeline/Preproc_Pipeline.dot')
scaffoldflow.write_graph(graph2use='colored', format='png', simple_form=True, 
                    dotfilename='../../Results/preproc_pipeline/Preproc_Pipeline_col.dot')

200813-10:57:23,918 nipype.workflow INFO:
	 Generated workflow graph: /media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/GITLAB/mind_blanking/Results/preproc_pipeline/Preproc_Pipeline.png (graph2use=orig, simple_form=True).
200813-10:57:24,218 nipype.workflow INFO:
	 Generated workflow graph: ../../Results/preproc_pipeline/Preproc_Pipeline_col.png (graph2use=colored, simple_form=True).


'../../Results/preproc_pipeline/Preproc_Pipeline_col.png'

## Running Pipeline

In [6]:
scaffoldflow.run('MultiProc', plugin_args={'n_procs': 8})

200813-10:57:25,366 nipype.workflow INFO:
	 Workflow scaffoldflow settings: ['check', 'execution', 'logging', 'monitoring']
200813-10:57:26,85 nipype.workflow INFO:
	 Running in parallel.
200813-10:57:26,88 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 13.98/13.98, Free processors: 8/8.
200813-10:57:26,189 nipype.workflow INFO:
	 [Node] Setting-up "scaffoldflow.selectfiles" in "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/Mind_Blanking/data/Neuroimaging/Raw_data/scaffoldflow/_subject_id_s8211/selectfiles".
200813-10:57:26,195 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
200813-10:57:26,202 nipype.workflow INFO:
	 [Node] Finished "scaffoldflow.selectfiles".
200813-10:57:28,95 nipype.workflow INFO:
	 [Job 0] Completed (scaffoldflow.selectfiles).
200813-10:57:28,97 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 13.98/13.98, Free processors: 8/8.


/home/sepehr/anaconda3/lib/python3.7/site-packages/nipype/algorithms/rapidart.py:443: RuntimeWarning: invalid value encountered in greater
  (np.nanmean(vol) / self.inputs.global_threshold)


200813-11:02:04,343 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 13.58/13.98, Free processors: 6/8.
                     Currently running:
                       * scaffoldflow.Functional.ArtifactDetection
                       * scaffoldflow.Structural.Segmentation
200813-11:02:04,474 nipype.workflow INFO:
	 [Node] Finished "scaffoldflow.Functional.ArtifactDetection".
200813-11:02:06,344 nipype.workflow INFO:
	 [Job 3] Completed (scaffoldflow.Functional.ArtifactDetection).
200813-11:02:06,346 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 13.78/13.98, Free processors: 7/8.
                     Currently running:
                       * scaffoldflow.Structural.Segmentation
200813-11:02:06,397 nipype.workflow INFO:
	 [Node] Setting-up "scaffoldflow.Denoising.MotionReg" in "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/Mind_Blanking/data/Neuroimaging/Raw_data/scaffoldflow/Denoising/_sub

200813-11:15:26,250 nipype.workflow INFO:
	 [Node] Finished "scaffoldflow.Functional.Normalization".
200813-11:15:27,121 nipype.workflow INFO:
	 [Job 8] Completed (scaffoldflow.Functional.Normalization).
200813-11:15:27,129 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 13.98/13.98, Free processors: 8/8.
200813-11:15:27,203 nipype.workflow INFO:
	 [Node] Setting-up "scaffoldflow.Denoising.Smoothing" in "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/Mind_Blanking/data/Neuroimaging/Raw_data/scaffoldflow/Denoising/_subject_id_s8211/Smoothing".
200813-11:15:27,203 nipype.workflow INFO:
	 [Node] Setting-up "scaffoldflow.Denoising.CSFReg" in "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/Projects/Mind_Blanking/data/Neuroimaging/Raw_data/scaffoldflow/Denoising/_subject_id_s8211/CSFReg".
200813-11:15:27,204 nipype.workflow INFO:
	 [Node] Setting-up "scaffoldflow.Denoising.WMReg" in "/media/sepehr/d504ddce-3b3c-4271-a3f5-2c6dd4c230f3/

/home/sepehr/anaconda3/lib/python3.7/site-packages/nipype/algorithms/confounds.py:1099: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  betas = np.linalg.lstsq(X, data.T)[0]


200813-11:15:29,124 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 0 jobs ready. Free memory (GB): 13.38/13.98, Free processors: 5/8.
                     Currently running:
                       * scaffoldflow.Denoising.WMReg
                       * scaffoldflow.Denoising.CSFReg
                       * scaffoldflow.Denoising.Smoothing


/home/sepehr/anaconda3/lib/python3.7/site-packages/nipype/algorithms/confounds.py:1099: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  betas = np.linalg.lstsq(X, data.T)[0]


200813-11:15:31,797 nipype.workflow INFO:
	 [Node] Finished "scaffoldflow.Denoising.CSFReg".
200813-11:15:33,124 nipype.workflow INFO:
	 [Job 12] Completed (scaffoldflow.Denoising.CSFReg).
200813-11:15:33,136 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 13.58/13.98, Free processors: 6/8.
                     Currently running:
                       * scaffoldflow.Denoising.WMReg
                       * scaffoldflow.Denoising.Smoothing
200813-11:15:40,413 nipype.workflow INFO:
	 [Node] Finished "scaffoldflow.Denoising.WMReg".
200813-11:15:41,133 nipype.workflow INFO:
	 [Job 17] Completed (scaffoldflow.Denoising.WMReg).
200813-11:15:41,140 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 13.78/13.98, Free processors: 7/8.
                     Currently running:
                       * scaffoldflow.Denoising.Smoothing
200813-11:15:41,207 nipype.workflow INFO:
	 [Node] Setting-up "scaffoldflow.Denoising.